# *CoastSat*: Grassy (Wave Swell Energy), King Island, Australia


21/3: Fixed transects using a radius-like orientation method. Finalised data production. Transect 67 is in the lee of the device. Seems to be some embayment rotation effects?  
17/3: Create images from download .TIF files and map shorelines. Tide correction performed. Need to get transects locations (every 50/100 m?).  
16/03/22: Download full Grassy Harbour embayment from Sentinel-2 inception date (23 June 2015) until present (16 March 2022).

## 1. Import Packages

In [1]:
import os
import numpy as np
import pickle
import warnings
warnings.filterwarnings("ignore")
import matplotlib
matplotlib.use('Qt5Agg')
import matplotlib.pyplot as plt
from matplotlib import gridspec
plt.ion()
import pandas as pd
from datetime import datetime
from coastsat import SDS_download, SDS_preprocess, SDS_shoreline, SDS_tools, SDS_transects

from ftplib import FTP
import pytz
import geopandas as gpd
from shapely.geometry import LineString


Bad key "text.kerning_factor" on line 4 in
C:\Users\s5245653\Anaconda3\envs\coastsat\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test_patch.mplstyle.
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.1.2/matplotlibrc.template
or from the matplotlib source distribution


## 2. Retrieval of the images from Google-Earth Engine

Define the region of interest (`polygon`), the date range (`dates`) and the satellite missions (`sat_list`) from which you wish to retrieve the satellite images. The images will be cropped on the Google Earth Engine server and only the region of interest will be downloaded as a .tif file. The files will stored in the directory defined in `filepath`. 

Make sure the area of your ROI is smaller than 100 km2 (i.e. 10 km by 10 km).

The function `SDS_download.check_images_available(inputs)` will print the number of images available for your inputs.

### 2.1 Set up parameters for image download

In [2]:
# region of interest (longitude, latitude)
polygon = [[[144.046,-40.0645],
            [144.061,-40.0645],
            [144.061,-40.0725],
            [144.046,-40.0725]]]
# it's recommended to convert the polygon to the smallest rectangle (sides parallel to coordinate axes)       
polygon = SDS_tools.smallest_rectangle(polygon)
# date range
dates = ['2015-06-23', '2022-03-16']
# satellite missions
sat_list = ['S2']
# name of the project folder
sitename = 'WSE_' + dates[0] + '_' + dates[1]
# directory where the data will be stored
filepath = os.path.join(os.getcwd(), 'data')
# put all the inputs into a dictionnary
inputs = {'polygon': polygon, 'dates': dates, 'sat_list': sat_list, 'sitename': sitename, 'filepath':filepath}

In [173]:
settings = { 
    # general parameters:
    'cloud_thresh': 0.5,        # threshold on maximum cloud cover
    'output_epsg': 3857,        # epsg code of spatial reference system desired for the output   
    # quality control:
    'check_detection': True,    # if True, shows each shoreline detection to the user for validation
    'adjust_detection': False,  # if True, allows user to adjust the postion of each shoreline by changing the threhold
    'save_figure': True,        # if True, saves a figure showing the mapped shoreline for each image
    # [ONLY FOR ADVANCED USERS] shoreline detection parameters:
    'min_beach_area': 4500,     # minimum area (in metres^2) for an object to be labelled as a beach
    'buffer_size': 150,         # radius (in metres) for buffer around sandy pixels considered in the shoreline detection
    'min_length_sl': 200,       # minimum length (in metres) of shoreline perimeter to be valid
    'cloud_mask_issue': False,  # switch this parameter to True if sand pixels are masked (in black) on many images  
    'sand_color': 'default',    # 'default', 'dark' (for grey/black sand beaches) or 'bright' (for white sand beaches)
    # add the inputs defined previously
    'inputs': inputs,
    # defines the along-shore distance over which to consider shoreline points to compute the median intersection (robust to outliers)
    'along_dist' : 40,
    'max_dist_ref' : 80 # max distance (in meters) allowed from the reference shoreline
}
# for tidal correction
beach_slope = 0.05
# for data cleaning
transect_threshold = 50

### 2.2 (optional) Check number of images available for parameters

In [35]:
SDS_download.check_images_available(inputs);

Images available between 2015-06-23 and 2022-03-16:
- In Landsat Tier 1 & Sentinel-2 Level-1C:
  S2: 1265 images
  Total: 1265 images


### 2.3 (optional) Download images from Google Earth Engine -- SLOW

In [37]:
# inputs['include_T2'] = True
metadata = SDS_download.retrieve_images(inputs)

Images available between 2015-06-23 and 2022-03-16:
- In Landsat Tier 1 & Sentinel-2 Level-1C:
  S2: 1265 images

KeyboardInterrupt: 

### 2.4 Load the metadata file
Do this instead of 2.2 and 2.3 once the images are already downloaded.

In [3]:
metadata = SDS_download.get_metadata(inputs) 

## 2. Shoreline extraction

This section maps the position of the shoreline on the satellite images. The user can define the cloud threhold (`cloud_thresh`) and select the spatial reference system in which to output the coordinates of the mapped shorelines (`output_epsg`). See http://spatialreference.org/ to find the EPSG number corresponding to your local coordinate system. Make sure that your are using cartesian coordinates and not spherical coordinates (lat,lon) like WGS84. If unsure, use 3857 which is the web mercator projection (used by Google Maps).

To quality control each shoreline detection and manually validate the mapped shorelines, the user has the option to set the parameter `check_detection` to **True**. 
To adjust the position of each shoreline by modifying the threshold defining the sand/water interface you can set `adjust_detection` to **True**. 
Finally, to save a figure for each mapped shoreline as a .jpg in the folder */jpg_files/detection* set `save_figure` to **True**. 

The other parameters are for advanced users only and are described in the README.

### [OPTIONAL] Save .jpg of the satellite images 
Saves .jpg files of the preprocessed satellite images (cloud masking + pansharpening/down-sampling) under *./data/sitename/jpeg_files\preprocessed*

In [5]:
SDS_preprocess.save_jpg(metadata, settings)

Satellite images saved as .jpg in C:\Users\s5245653\OneDrive - Griffith University\Projects\CoastSat\data\WSE_2015-06-23_2022-03-16\jpg_files\preprocessed


### [OPTIONAL] Digitize a reference shoreline
Creates a reference shoreline which helps to identify outliers and false detections. The reference shoreline is manually digitised by the user on one of the images. The parameter `max_dist_ref` defines the maximum distance from the reference shoreline (in metres) at which a valid detected shoreline can be. If you think that the default value of 100 m will not capture the full shoreline variability of your site, increase this value to an appropriate distance.

In [6]:
%matplotlib qt
settings['reference_shoreline'] = SDS_preprocess.get_reference_sl(metadata, settings)

Reference shoreline has been saved in C:\Users\s5245653\OneDrive - Griffith University\Projects\CoastSat\data\WSE_2015-06-23_2022-03-16


### Batch shoreline detection
Extracts the 2D shorelines from the images in the spatial reference system specified by the user in `'output_epsg'`. The mapped shorelines are saved into `output.pkl` (under *./data/sitename*) and `output.geojson` (to be used in a GIS software).

If you see that the sand pixels on the images are not being identified, change the parameter `sand_color` from `default` to `dark` or `bright` depending on the color of your beach. 

In [7]:
%matplotlib qt
output = SDS_shoreline.extract_shorelines(metadata, settings)

Mapping shorelines:
S2:   100%


Then remove duplicates and images with inaccurate georeferencing (threhsold at 10m)

In [8]:
output = SDS_tools.remove_duplicates(output) # removes duplicates (images taken on the same date by the same satellite)
output = SDS_tools.remove_inaccurate_georef(output, 10) # remove inaccurate georeferencing (set threshold to 10 m)

0 duplicates
3 bad georef


For use in GIS applications, you can save the mapped shorelines as a GEOJSON layer which can be easily imported into QGIS for example. You can choose to save the shorelines as a collection of lines or points (sometimes the lines are crossing over so better to use points).

In [10]:
geomtype = 'lines' # choose 'points' or 'lines' for the layer geometry
gdf = SDS_tools.output_to_gdf(output, geomtype)
gdf.crs = {'init':'epsg:'+str(settings['output_epsg'])} # set layer projection
# save GEOJSON layer to file
gdf.to_file(os.path.join(inputs['filepath'], inputs['sitename'], '%s_output_%s.geojson'%(sitename,geomtype)),
                                driver='GeoJSON', encoding='utf-8')

Simple plot of the mapped shorelines. The coordinates are stored in the output dictionnary together with the exact dates in UTC time, the georeferencing accuracy and the cloud cover.

In [11]:
fig = plt.figure(figsize=[15,8])
plt.axis('equal')
plt.xlabel('Eastings')
plt.ylabel('Northings')
plt.grid(linestyle=':', color='0.5')
for i in range(len(output['shorelines'])):
    sl = output['shorelines'][i]
    date = output['dates'][i]
    plt.plot(sl[:,0], sl[:,1], '.', label=date.strftime('%d-%m-%Y'))
plt.legend();

## 3. Shoreline analysis

In this section we show how to compute time-series of cross-shore distance along user-defined shore-normal transects.

**If you have already mapped the shorelines**, just load the output file (`output.pkl`) by running the section below

In [4]:
filepath = os.path.join(inputs['filepath'], sitename)
with open(os.path.join(filepath, sitename + '_output' + '.pkl'), 'rb') as f:
    output = pickle.load(f)

There are 3 options to define the coordinates of the shore-normal transects:

**Option 1**: the user can interactively draw the shore-normal transects along the beach by calling:

In [ ]:
%matplotlib qt
transects = SDS_transects.draw_transects(output, settings)

**Option 2**: the user can load the transect coordinates (make sure the spatial reference system is the same as defined previously by the parameter *output_epsg*) from a .geojson file by calling:

In [5]:
geojson_file = os.path.join(os.getcwd(), 'data', sitename, 'Transects_WSE.geojson')
gdf = gpd.read_file(geojson_file)

#geojson_file = os.path.join(os.getcwd(), 'data', sitename, 'Transects_WSE.geojson')
#transects = SDS_tools.transects_from_geojson(geojson_file)

In [95]:
no_transects = 72
transects = dict([])
blank = []
coord_for_dist = [16036074.0,-4876017.0] # a location at the centre of the bay to use to correct orientation of the transects
for i in range(no_transects-4): # first 4 transects were "bad"
    # load transect data from geopandas dataframe into an array
    a = np.array(list(LineString(gdf['geometry'][i+4]).coords))
    # Create temporary variable to use to correct transect orientation
    temp = a - coord_for_dist 
    temp = [np.linalg.norm(i) for i in temp]
    loc_max = np.argmax(temp)
    loc_min = np.argmin(temp)
    # Correct transect orientation
    temp_1 = a[loc_max],a[loc_min]
    temp_2 = np.array(temp_1)
    # Add to transect dictionary
    transects[str(i)] = temp_2

**Option 3**: manually provide the coordinates of the transects as shown in the example below:

In [6]:
# To transform lat/lon to eastings/northings (EPSG 3857) use https://epsg.io/transform
# location of WEC (approx.): 16036531, -4875504
transects = dict([])
transects['WSE'] = np.array([[16036540,-4875320], [16036537,-4875529]])

Plot the location of the transects, make sure they are in the right location with the origin always landwards!

In [96]:
fig = plt.figure(figsize=[15,8], tight_layout=True)
plt.axis('equal')
plt.xlabel('Eastings')
plt.ylabel('Northings')
plt.grid(linestyle=':', color='0.5')
for i in range(len(output['shorelines'])):
    sl = output['shorelines'][i]
    date = output['dates'][i]
    plt.plot(sl[:,0], sl[:,1], '.', label=date.strftime('%d-%m-%Y'))
for i,key in enumerate(list(transects.keys())):
    plt.plot(transects[key][0,0],transects[key][0,1], 'bo', ms=5)
    plt.plot(transects[key][:,0],transects[key][:,1],'k-',lw=1)
    plt.text(transects[key][0,0]-100, transects[key][0,1]+100, key,
                va='center', ha='right', bbox=dict(boxstyle="square", ec='k',fc='w'))

Now, intersect the transects with the 2D shorelines to obtain time-series of cross-shore distance.

The time-series of shoreline change for each transect are saved in a .csv file in the data folder (all dates are in UTC time). 

In [97]:
cross_distance = SDS_transects.compute_intersection(output, transects, settings)

Time-series of the shoreline change along the transects saved as:
C:\Users\s5245653\OneDrive - Griffith University\Projects\CoastSat\data\WSE_2015-06-23_2022-03-16\transect_time_series.csv


Plot the time-series of shoreline change along each transect

In [98]:
fig = plt.figure(figsize=[15,8], tight_layout=True)
gs = gridspec.GridSpec(len(cross_distance),1)
gs.update(left=0.05, right=0.95, bottom=0.05, top=0.95, hspace=0.05)
for i,key in enumerate(cross_distance.keys()):
    if np.all(np.isnan(cross_distance[key])):
        continue
    ax = fig.add_subplot(gs[i,0])
    ax.grid(linestyle=':', color='0.5')
    ax.set_ylim([-50,50])
    ax.plot(output['dates'], cross_distance[key]- np.nanmedian(cross_distance[key]), '-o', ms=6, mfc='w')
    ax.set_ylabel('distance [m]', fontsize=12)
    ax.text(0.5,0.95, key, bbox=dict(boxstyle="square", ec='k',fc='w'), ha='center',
            va='top', transform=ax.transAxes, fontsize=14)

## 4. Tidal correction

This last section shows how to tidally-correct the time-series of shoreline change using time-series of tide level and an estimate of the beach slope.

For this example, measured water levels for Sydney are stored in a csv file located [here](https://github.com/kvos/CoastSat/blob/master/examples/NARRA_tides.csv). When using your own file make sure that the dates are in UTC time, as the CoastSat shorelines are also in UTC, and the datum for the water levels is approx. Mean Sea Level.

We assume that the beach slope at Narrabeen-Collaroy is 0.1 along all transects.

**Note**: if you don't have measured water levels and beach slope, it is possible to obtain an estimate of the beach slope and time-series of modelled tide levels at the time of image acquisition from the [FES2014](https://www.aviso.altimetry.fr/es/data/products/auxiliary-products/global-tide-fes/description-fes2014.html) global tide model by using the [CoastSat.slope](https://github.com/kvos/CoastSat.slope) repository (see [this publication](https://doi.org/10.1029/2020GL088365) for more details, open acess preprint [here](https://www.essoar.org/doi/10.1002/essoar.10502903.1)). Instructions on how to install the global tide model are available [here](https://github.com/kvos/CoastSat.slope/blob/master/doc/FES2014_installation.md).

### Assess tide times of sat image

In [118]:
# load the tide data (FES)
tide_filepath = 'fes-2.9.3-Source/data/tide_data_15_22.csv'
tide_data = pd.read_csv(tide_filepath, parse_dates=['dates'])
dates_ts = [_.to_pydatetime() for _ in tide_data['dates']]
dates_ts = [pytz.utc.localize(d) for d in dates_ts] # line added to include timezone
tides_ts = np.array(tide_data['tide'])

# get tide levels corresponding to the time of image acquisition
dates_sat = output['dates']
tides_sat = SDS_tools.get_closest_datapoint(dates_sat, dates_ts, tides_ts)

Extracting closest points: 100%

In [119]:
# plot the subsampled tide data
fig, ax = plt.subplots(1,1,figsize=(15,4), tight_layout=True)
ax.grid(which='major', linestyle=':', color='0.5')
ax.plot(tide_data['dates'], tide_data['tide'], '-', color='0.6', label='all time-series')
ax.plot(dates_sat, tides_sat, '-o', color='k', ms=6, mfc='w',lw=1, label='image acquisition')
ax.set(ylabel='tide level [m]',xlim=[dates_sat[0],dates_sat[-1]], title='Water levels at the time of image acquisition');
ax.legend();

Apply tidal correction using a linear slope and a reference elevation to which project all the time-series of cross-shore change (to get time-series at Mean Sea Level, set `reference elevation` to 0. You also need an estimate of the beach slope. 

### Perform Tidal Correction
Note beach slope parameter value

In [191]:
# tidal correction along each transect
reference_elevation = 0 # elevation at which you would like the shoreline time-series to be
cross_distance_tidally_corrected = {}
for key in cross_distance.keys():
    correction = (tides_sat-reference_elevation)/beach_slope
    cross_distance_tidally_corrected[key] = cross_distance[key] + correction
# add line to subtract the median from the distance
    cross_distance_tidally_corrected[key] = cross_distance_tidally_corrected[key] - np.nanmedian(cross_distance_tidally_corrected[key])
# store the tidally-corrected time-series in a .csv file
out_dict = dict([])
out_dict['dates'] = dates_sat
for key in cross_distance_tidally_corrected.keys():
    out_dict['Transect '+ key] = cross_distance_tidally_corrected[key]
df = pd.DataFrame(out_dict)
# tidy up dataframe
df = df.set_index('dates')
df[np.abs(df) > transect_threshold] = np.nan # remove outliers
# Create csv file
fn = os.path.join(settings['inputs']['filepath'],settings['inputs']['sitename'],
                  'transect_time_series_tide_corrected.csv')
df.to_csv(fn, sep=',')
print('Tidally-corrected time-series of the shoreline change along the transects saved as:\n%s'%fn)

Tidally-corrected time-series of the shoreline change along the transects saved as:
C:\Users\s5245653\OneDrive - Griffith University\Projects\CoastSat\data\WSE_2015-06-23_2022-03-16\transect_time_series_tide_corrected.csv


In [102]:
# plot the time-series of shoreline change (both raw and tidally-corrected)
fig = plt.figure(figsize=[15,8], tight_layout=True)
gs = gridspec.GridSpec(len(cross_distance),1)
gs.update(left=0.05, right=0.95, bottom=0.05, top=0.95, hspace=0.05)
for i,key in enumerate(cross_distance.keys()):
    if np.all(np.isnan(cross_distance[key])):
        continue
    ax = fig.add_subplot(gs[i,0])
    ax.grid(linestyle=':', color='0.5')
    ax.set_ylim([-50,50])
    ax.plot(output['dates'], cross_distance[key]- np.nanmedian(cross_distance[key]), '-o', ms=6, mfc='w', label='raw')
    ax.plot(output['dates'], cross_distance_tidally_corrected[key]- np.nanmedian(cross_distance[key]), '-o', ms=6, mfc='w', label='tidally-corrected')
    ax.set_ylabel('distance [m]', fontsize=12)
    ax.text(0.5,0.95, key, bbox=dict(boxstyle="square", ec='k',fc='w'), ha='center',
            va='top', transform=ax.transAxes, fontsize=14)
ax.legend();
#plt.savefig('raw_vs_tidecorrected_transects.png')

In [193]:
plot_name = 'data/WSE_2015-06-23_2022-03-16/Transect WSE Tide Edit'
transect_name = 'Transect 67'

fig, ax = plt.subplots(1,1,figsize=(15,8), tight_layout=True)
ax.grid(which='major', linestyle=':', color='0.5')
ax.plot(df.index.values, 
        df[transect_name],
        'o-', color='mediumblue', lw=1.5)

ax.set(ylabel='shoreline position [m]', title='Tide-Corrected Shoreline Time Series from transect in lee of WEC');
# Add line for WEC installation date
ax.axvline(x='2021-01-10', ymin=0.1, ymax=0.9, color='black', ls='--', lw=1, label='Uniwave200 Install Date')
ax.legend(loc='lower right')

#plt.savefig(plot_name)

### [EXTRA] FTP From FES2014 (AVISO+)

In [ ]:
#### FTP setup and file/folder list
ftp = FTP("ftp-access.aviso.altimetry.fr")
ftp.login(user='james.thompson3@griffithuni.edu.au', passwd = 'o3KUN5')
ftp.cwd('/auxiliary/tide_model/fes2014_elevations_and_load') # need to add folder directory to the end of this url
ftp.retrlines('LIST') 

#### File retrieval
filename = 'ocean_tide_extrapolated.tar.xz.sha256sum'
retrieval_name = 'RETR ' + filename
with open(filename, 'wb') as fp:
    ftp.retrbinary(retrieval_name, fp.write)

ftp.close()

#### Now go to tides environment in Anaconda to create the Tide data file using other notebook